In [55]:
!wget = q https://gbcdn.mrgcdn.ru/uploads/asset/5669369/attachment/8d87d92d70004b56a3efbe60485c3e4d.ipynb

--2023-11-17 15:24:21--  http://=/
Resolving = (=)... failed: Name or service not known.
wget: unable to resolve host address ‘=’
--2023-11-17 15:24:21--  http://q/
Resolving q (q)... failed: Name or service not known.
wget: unable to resolve host address ‘q’
--2023-11-17 15:24:21--  https://gbcdn.mrgcdn.ru/uploads/asset/5669369/attachment/8d87d92d70004b56a3efbe60485c3e4d.ipynb
Resolving gbcdn.mrgcdn.ru (gbcdn.mrgcdn.ru)... 5.181.61.0
Connecting to gbcdn.mrgcdn.ru (gbcdn.mrgcdn.ru)|5.181.61.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 496677 (485K) [application/json]
Saving to: ‘8d87d92d70004b56a3efbe60485c3e4d.ipynb.1’

8d87d92d70004b56a3e 100%[===================>] 485.04K   871KB/s    in 0.6s    

2023-11-17 15:24:22 (871 KB/s) - ‘8d87d92d70004b56a3efbe60485c3e4d.ipynb.1’ saved [496677/496677]

FINISHED --2023-11-17 15:24:22--
Total wall clock time: 1.6s
Downloaded: 1 files, 485K in 0.6s (871 KB/s)


In [56]:
!pip install opendatasets

In [57]:
import opendatasets as od

In [58]:
import pandas as pd

In [59]:
import time

In [60]:
from pathlib import Path

In [61]:
dataset_path = Path('imdb-user-reviewa', 'song_lyrics.csv')
if not dataset_path.is_file():
  od.download('https://www.kaggle.com/datasets/sadmadlad/imdb-user-reviews')

Skipping, found downloaded files in "./imdb-user-reviews" (use force=True to force download)


In [62]:
df = pd.read_csv('/content/IMDB.csv')
df.head()

,Name,Year,Episodes,Type,Rating,Image-src,Description,Name-href
0,1. Breaking Bad,2008–2013,62 eps,TV-MA,9.5,https://m.media-amazon.com/images/M/MV5BYmQ4YW...,A chemistry teacher diagnosed with inoperable ...,https://www.imdb.com/title/tt0903747/?ref_=cht...
1,2. Planet Earth II,2016,6 eps,TV-G,9.5,https://m.media-amazon.com/images/M/MV5BMGZmYm...,David Attenborough returns with a new wildlife...,https://www.imdb.com/title/tt5491994/?ref_=cht...
2,3. Planet Earth,2006,11 eps,TV-PG,9.4,https://m.media-amazon.com/images/M/MV5BMzMyYj...,A documentary series on the wildlife found on ...,https://www.imdb.com/title/tt0795176/?ref_=cht...
3,4. Band of Brothers,2001,10 eps,TV-MA,9.4,https://m.media-amazon.com/images/M/MV5BMTI3OD...,The story of Easy Company of the U.S. Army 101...,https://www.imdb.com/title/tt0185906/?ref_=cht...
4,5. Chernobyl,2019,5 eps,TV-MA,9.4,https://m.media-amazon.com/images/M/MV5BNTdkN2...,"In April 1986, an explosion at the Chernobyl n...",https://www.imdb.com/title/tt7366338/?ref_=cht...


In [63]:
def find_longest_string(list_of_strings):
    longest_string = None
    longest_string_len = 0
    for s in list_of_strings:
        # оставляем только буквы
        s = ''.join(x for x in s if x.isalpha())
        s_len = len(s)
        if s_len > longest_string_len:
            longest_string_len = s_len
            longest_string = s
    return longest_string, longest_string_len

In [64]:
list_of_strings = ['abc123', 'python', 'sasha2000']
print(find_longest_string(list_of_strings))

('python', 6)


In [65]:
large_list_of_strings = list_of_strings * 1_000_000

In [66]:
%time print(find_longest_string(large_list_of_strings))

('python', 6)
CPU times: user 3.4 s, sys: 14.6 ms, total: 3.41 s
Wall time: 3.59 s


In [67]:
from functools import reduce

def chunkify(list_of_strings, number_of_chunks=30):
    step = len(list_of_strings) // number_of_chunks
    if step != 0:
        for i in range(0, len(list_of_strings), step):
            yield list_of_strings[i : i + step]
    else:
        yield list_of_strings

def chunks_mapper(chunk):
    return find_longest_string(chunk)

def reducer(p, c):
    if p[1] > c[1]:
        return p
    return c

In [68]:
%%timeit
data_chunks = chunkify(large_list_of_strings, number_of_chunks=1)

mapped = map(chunks_mapper, data_chunks)
reduced = reduce(reducer, mapped)
print(reduced)

('python', 6)
('python', 6)
('python', 6)
('python', 6)
('python', 6)
('python', 6)
('python', 6)
('python', 6)
4.22 s ± 2.29 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [69]:
from joblib import Parallel, delayed

In [70]:
%%timeit
data_chunks = chunkify(large_list_of_strings, number_of_chunks=2)

mapped = Parallel(n_jobs=2)(delayed(chunks_mapper)(chunk) for chunk in data_chunks)
reduced = reduce(reducer, mapped)

3.45 s ± 570 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [71]:
mapper = len

def chunks_mapper(chunk):
    mapped_chunk = map(mapper, chunk)
    mapped_chunk = zip(chunk, mapped_chunk)
    return reduce(reducer, mapped_chunk)

In [72]:
%%time
data_chunks = chunkify(large_list_of_strings, number_of_chunks=1)

mapped = map(chunks_mapper, data_chunks)
reduced = reduce(reducer, mapped)
print(reduced)

('sasha2000', 9)
CPU times: user 440 ms, sys: 1.06 ms, total: 441 ms
Wall time: 442 ms


In [73]:
%%time
from multiprocessing import Pool

data_chunks = chunkify(large_list_of_strings, number_of_chunks=2)
with Pool(2) as p:
    mapped = p.map(chunks_mapper, data_chunks)
    reduced = reduce(reducer, mapped)

CPU times: user 126 ms, sys: 47.2 ms, total: 173 ms
Wall time: 676 ms


In [74]:
%%time
# либо так:
data_chunks = chunkify(large_list_of_strings, number_of_chunks=2)
mapped = Parallel(n_jobs=2)(delayed(chunks_mapper)(chunk) for chunk in data_chunks)
reduced = reduce(reducer, mapped)

CPU times: user 118 ms, sys: 3.68 ms, total: 121 ms
Wall time: 602 ms


In [75]:
%time max_length = max(large_list_of_strings, key=len)

CPU times: user 154 ms, sys: 0 ns, total: 154 ms
Wall time: 161 ms


In [76]:
import json


n, mean, M2 = 0, 0.0, 0
for path in Path('imdb-user-reviews').glob('**/*'):
    if path.is_file() and path.suffix == '.json':
        with open(path, 'r') as f:
            info = json.load(f)
        score = float(info['movieIMDbRating'])
        n += 1
        delta = score - mean
        mean += delta / n
        M2 += delta * (score - mean)

print(mean, (M2 / n) ** (1/2))

8.03 1.051712888577486


In [77]:
def mapper(path):
    if path.is_file() and path.suffix == '.json':
        with open(path, 'r') as f:
            info = json.load(f)
        return (float(info['movieIMDbRating']), )


def reducer(score_data1, score_data2):
    if score_data1 is None and score_data2 is None:
        return None
    elif score_data1 is None:
        return score_data2
    elif score_data2 is None:
        return score_data1
    else:
        scores = []
        if len(score_data1) == 1:
            n, mean, M2 = 0, 0.0, 0
            scores.append(score_data1[0])
        else:
            n, mean, M2 = score_data1
        scores.append(score_data2[0])
        for score in scores:
            n += 1
            delta = score - mean
            mean += delta / n
            M2 += delta * (score - mean)
        return n, mean, M2

In [78]:
%%time
n, mean, M2 = reduce(reducer, map(mapper, Path('imdb-user-reviews').glob('**/*')))
print(mean, (M2 / n) ** (1/2))

8.03 1.051712888577486
CPU times: user 3.24 ms, sys: 27 µs, total: 3.27 ms
Wall time: 6.28 ms


In [79]:
%%time
n, mean, M2 = reduce(reducer, Parallel(n_jobs=2)(delayed(mapper)(path) for path in Path('imdb-user-reviews').glob('**/*')))
print(mean, (M2 / n) ** (1/2))

8.03 1.051712888577486
CPU times: user 26.6 ms, sys: 1.99 ms, total: 28.6 ms
Wall time: 50.1 ms


In [80]:
%%time
n, mean, M2 = reduce(reducer, map(mapper, list(Path('imdb-user-reviews').glob('**/*')) * 100_000))
print(mean, (M2 / n) ** (1/2))

8.030000000000062 1.0517128885775677
CPU times: user 32.7 s, sys: 14.8 s, total: 47.5 s
Wall time: 51.7 s


In [81]:
%%time
n, mean, M2 = reduce(reducer, Parallel(n_jobs=2)(delayed(mapper)(path) for path in list(Path('imdb-user-reviews').glob('**/*')) * 100_000))
print(mean, (M2 / n) ** (1/2))

8.030000000000062 1.0517128885775677
CPU times: user 31.1 s, sys: 405 ms, total: 31.5 s
Wall time: 1min 2s
